In [1]:
import numpy as np
import torch
import open3d as o3d

import sys
sys.path.append("..")

from vehicle_reconstruction import *
from vehicle_reconstruction.scene_segmentation import *
from open3d_vis_utils import draw_scenes

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Loading Point Cloud

In [2]:
pts = np.load("../data/00043_pts.npy").reshape(-1, 4).astype(np.float32)
gt_box = np.load("../data/00043_gt.npy").reshape(-1, 7).astype(np.float32)

print(pts.shape)
print(gt_box.shape)

(162103, 4)
(72, 7)


## Visualizaing Point Cloud

In [3]:
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="view")

draw_scenes(vis, points=pts, gt_boxes=gt_box, draw_origin=True)

vis.run()
vis.destroy_window()

## Preparing vehicle reconstructor

In [4]:
from vehicle_reconstruction import vehicle, vehicle_reconstructor, constants, quantify_bbox
from vehicle_reconstruction.data_utils import apolloscape_dataset

constants.apolloscape_constant.set_apolloscape_dir("/mnt/sda/uzuki_space/voxeltorch/assets/apollo_scape")
apolloscape = apolloscape_dataset()


loading 79 car models


In [5]:
TSDF_UNIT = torch.Tensor([0.1]*3).to(DEVICE)
bbox = apolloscape.get_bbox().to(DEVICE)
antisotropic_res = (bbox / TSDF_UNIT).ceil().int()
quantified_bbox = quantify_bbox(bbox, TSDF_UNIT)

apolloscape_meshes = apolloscape.get_batch_centered_meshes(quantified_bbox).to(DEVICE)

vehi_reconstructor = vehicle_reconstructor(resolution=antisotropic_res + 1, 
                      bbox=quantified_bbox,
                      unit=TSDF_UNIT)

if os.path.exists("../data/vehicle_ckpt.pt"):
    vehi_reconstructor.load_parameters("../data/vehicle_ckpt.pt")
else:
    print(f"Pretrained parameter not found!")
    vehi_reconstructor.fit_meshes(apolloscape_meshes)
    # vehi_reconstructor.save_parameters("../data/vehicle_ckpt.pt")


Pretrained parameter not found!


In [6]:
scene = point_cloud_scene(pts=torch.from_numpy(pts).to(DEVICE),
                          gt_boxes=torch.from_numpy(gt_box).to(DEVICE))
vehicles: vehicle_object = scene.get_vehicles()
print(vehicles.__len__())

72


In [ ]:
scene.pose_estimate(vehi_reconstructor=vehi_reconstructor, iter=20)

training_point_clouds = vehicles.get_training_point_clouds()
print(training_point_clouds.points_packed().min(dim=0))
print(training_point_clouds.points_packed().max(dim=0))

vis = o3d.visualization.Visualizer()
vis.create_window(window_name="view")
draw_scenes(vis, points=pts, gt_boxes=gt_box,
            draw_origin=True)

mesh = vehicles.reconstruct_at_scene(
    vehi_reconstructor, threshold=10, show_rooftop=True)

mesh = o3d.geometry.TriangleMesh(
    o3d.utility.Vector3dVector(mesh.verts_packed().detach().cpu().numpy()),
    o3d.utility.Vector3iVector(mesh.faces_packed().detach().cpu().numpy()))

# vertex_colors = np.ones_like(mesh.vertices) * np.array([0.7, 0.75, 0.7])
# vertex_colors[rooftop_idx] = np.array([0.0, 0.6, 0.0])
# vertex_colors = o3d.utility.Vector3dVector(vertex_colors)
# mesh.vertex_colors = vertex_colors
# # mesh = mesh.filter_smooth_simple(number_of_iterations=5)
mesh.compute_vertex_normals()

vis.add_geometry(mesh)

# for vehi in vehicles:
#     if vehi is None:
#         continue
    
#     mesh = vehi.reconstruct_at_scene(
#         vehi_reconstructor, show_rooftop=True)
    
#     # rooftop_center = mesh.vertices[rooftop_idx].mean(0)
    
#     # axis_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(
#     #         size=1.0, origin=rooftop_center)
#     # vis.add_geometry(axis_pcd)
    
#     mesh = o3d.geometry.TriangleMesh(
#         o3d.utility.Vector3dVector(mesh.verts_packed().detach().cpu().numpy()),
#         o3d.utility.Vector3iVector(mesh.faces_packed().detach().cpu().numpy()))
    

#     # vertex_colors = np.ones_like(mesh.vertices) * np.array([0.7, 0.75, 0.7])
#     # vertex_colors[rooftop_idx] = np.array([0.0, 0.6, 0.0])
#     # vertex_colors = o3d.utility.Vector3dVector(vertex_colors)
#     # mesh.vertex_colors = vertex_colors
#     # # mesh = mesh.filter_smooth_simple(number_of_iterations=5)
#     mesh.compute_vertex_normals()

#     vis.add_geometry(mesh)
vis.run()
vis.destroy_window()

Time cost:0.22104620933532715, 0.008841848373413086 per epoch
torch.return_types.min(
values=tensor([-1.0091e+02, -9.9274e+01,  1.3037e-03], device='cuda:0'),
indices=tensor([16896, 15872, 34313], device='cuda:0'))
torch.return_types.max(
values=tensor([118.5135, 105.1639,   1.9644], device='cuda:0'),
indices=tensor([20480, 13312, 17416], device='cuda:0'))


In [8]:

# centroid_point_clouds = vehicles.get_training_point_clouds()

# for pts in centroid_point_clouds:
#     vis = o3d.visualization.Visualizer()
#     vis.create_window(window_name="view")
    
#     # pts_centroid, bbox_centroid = vehi.centroid()
#     # point_colors = np.zeros((pts_centroid.shape[0], 3))
#     # point_colors[:, 0] = 1.0
#     # pts:Pointclouds = pts
#     draw_scenes(vis, points=pts.points_packed(), draw_origin=True)
#     # draw_scenes(vis, points=pts_centroid,
#     #             gt_boxes=bbox_centroid.unsqueeze(dim=0), draw_origin=True, point_colors=point_colors)
    
    
    
#     # vehi.reconstruct_vehicle(vehi_reconstructor)
#     # vehicle_mesh: vehicle = vehi.vehicle
#     # vehicle_mesh = vehicle_mesh.to_trimesh()
#     # vehicle_mesh.vertices *= 0.1
#     # vehicle_mesh.vertices = vehicle_mesh.vertices[:, [2, 0, 1]]
    
#     # vehicle_mesh.vertices -= np.array([standard_bbox[2]/2.0,
#     #                                   standard_bbox[0]/2.0,
#     #                                   bbox_centroid[5].cpu() / 2.0])
    
#     # mesh = o3d.geometry.TriangleMesh(
#     #     o3d.utility.Vector3dVector(vehicle_mesh.vertices), 
#     #     o3d.utility.Vector3iVector(vehicle_mesh.faces))
#     # # mesh = mesh.filter_smooth_simple(number_of_iterations=1)
#     # mesh.compute_vertex_normals()
    
#     # vis.add_geometry(mesh)
#     vis.run()
#     vis.destroy_window()

{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 69.118263244628906, 39.679920196533203, 16.415634155273438 ],
			"boundingbox_min" : [ -0.059999999999999998, -39.679874420166016, -6.9146575927734375 ],
			"field_of_view" : 60.0,
			"front" : [ -0.90307097537632919, 0.0017988087570628851, 0.42948757574567964 ],
			"lookat" : [ 34.529131622314452, 2.288818359375e-05, 4.75048828125 ],
			"up" : [ 0.42948904059539766, 0.0070563614983622357, 0.90304450154510629 ],
			"zoom" : 0.69999999999999996
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}